# Filters

This example shows how to control map filters and listen to map filter change events with Unfolded Map SDK.

Let's again create a local map and add data to it:

In [ ]:
from unfolded.map_sdk import UnfoldedMap
unfolded_map = UnfoldedMap()

In [ ]:
from sidecar import Sidecar
sc = Sidecar(title='Unfolded map', anchor='split-right')
with sc:
    display(unfolded_map)

In [ ]:
import pandas as pd
df = pd.read_csv('./data/earthquakes.csv')

In [ ]:
unfolded_map.add_dataset({
    'label': 'Earthquakes',
    'data': df
})

## Adding a filter

Say, we want to filter the data points by the `Magnitude` column. Let's first find out what's the extent of the values in this column:

In [ ]:
magnitude_extent = [df['Magnitude'].min(), df['Magnitude'].max()]
magnitude_extent

Now we can set the filter to only show the points in the top half of the range:

In [ ]:
unfolded_map.set_filter({
    'id': 'magnitude_filter',
    'field': 'Magnitude',
    'value': [
        (magnitude_extent[1] + magnitude_extent[0])/2,
        magnitude_extent[1]
    ]
})

Or the bottom half:

In [ ]:
unfolded_map.set_filter({
    'id': 'magnitude_filter',
    'field': 'Magnitude',
    'value': [
        magnitude_extent[0],
        (magnitude_extent[1] + magnitude_extent[0])/2
    ]
})

## Controlling the filter from the notebook

We can use the range slider from `ipywidgets` to control the filter in the map. First, we create the slider:

In [ ]:
import ipywidgets as widgets

slider = widgets.FloatRangeSlider(
    value=magnitude_extent,
    min=magnitude_extent[0],
    max=magnitude_extent[1],
    step=0.1,
    description='Magnitude:',
    continuous_update=True
)

Here we create an event handler to listen to the slider change events:

In [ ]:
def update_value_filter(change):
    if 'new' in change and 'value' in change['new']:
        unfolded_map.set_filter({
            'id': 'magnitude_filter',
            'field': 'Magnitude',
            'value': change['new']['value']
        })

In [ ]:
slider.observe(update_value_filter)
display(slider)

Now try moving the slider. You should see the changes applied to the map.

The following will unregister the observer:

In [ ]:
slider.unobserve(None)

## Syncing the slider in the notebook with the map filter

We can register [event handlers](https://docs.unfolded.ai/map-sdk/api#events) to be notified of filter changes in the map. Here's how we can syncronize the above range slider with the Magnitude filter in the map:

In [ ]:
def on_filter_sync(info):
    if 'Magnitude' in info['name'] and info['prop'] == 'value':
        slider.value = info['value']

unfolded_map.set_map_event_handlers({
    'on_filter': on_filter_sync
})

Now try changing the "Magnitude" filter in the "Filter" pane of the left sidebar in the map. You should see the slider above in the notebook update. 

##  Debugging an event handler

Here's how you can output the event info when filter events are triggered:

In [ ]:
import ipywidgets as widgets
output = widgets.Output()
@output.capture(clear_output=True)
def on_filter_output(info):
    print(info)
output

In [ ]:
unfolded_map.set_map_event_handlers({
    'on_filter': on_filter_output
})